In [27]:
import json
import json_repair
import os
import pandas as pd
from typing import Dict, List, Optional
from tqdm.auto import tqdm

In [ ]:
from openai import OpenAI
VSE_GPT_API_KEY = ""

client = OpenAI(
    api_key=VSE_GPT_API_KEY,
    base_url="https://api.vsegpt.ru/v1",
)

# Data

In [3]:
with open('/Users/alfa/Code/financial_assistant/data/interim/test_qual_investor/questions.json', 'r') as f:
    questions = json.load(f)

In [4]:
questions[0]

{'id': '1.4',
 'question': 'Ликвидность акции характеризует',
 'options': [{'letter': 'А',
   'option_text': 'Способность инвестора продать акцию с минимальными для него потерями в минимальный срок.'},
  {'letter': 'Б',
   'option_text': 'Разницу цены такой акции на разных торговых площадках.'},
  {'letter': 'В',
   'option_text': 'Вероятность погашения акции компанией – эмитентом.'},
  {'letter': 'Г',
   'option_text': 'Ни один из ответов не является правильным.'}],
 'answer': 'А',
 'chapter': 1}

In [5]:
with open('/Users/alfa/Code/financial_assistant/data/interim/test_qual_investor/chapters.json', 'r') as f:
    chapters = json.load(f)

In [6]:
chapters

{'1': 'Покупка иностранных акций',
 '2': 'Акции, не включенные в котировальные списки',
 '3': 'Допуск к необеспеченным сделкам (маржинальная торговля)',
 '4': 'Заключение договоров РЕПО',
 '5': 'Опционы, фьючерсы, производные финансовые инструменты',
 '6': 'Структурные облигации',
 '7': 'Паи закрытых паевых инвестиционных фондов (ЗПИФ)',
 '8': 'Облигации российских эмитентов, которым не присвоен рейтинг или он ниже уровня',
 '9': 'Облигации иностранных эмитентов в валюте (еврооблигации) которым не присвоен рейтинг или он ниже нужного уровня',
 '10': 'Облигации со структурным доходом',
 '11': 'Вопросы для допуска к иностранным ETF'}

# Zero-shot

## Prompt

In [7]:
with open("/Users/alfa/Code/financial_assistant/artifacts/prompts/system_v1.md", "r") as f:
    system_prompt = f.read()

print(system_prompt)

You are qualified financial and investment assistant. 
Provide helpful answers to any question.  
Stricly follow user instructions.  


In [8]:
with open("/Users/alfa/Code/financial_assistant/artifacts/prompts/v1.md", "r") as f:
    prompt_template = f.read()

In [9]:
print(prompt_template)

### Instructions ###

Answer the multiple-choice ###Question### about Russian invest market.
Follow ###Asnwer Format###.


### Answer Format ###
{{
    "reasoning" : "provide your brief (1-2 sentences) reasoning here",
    "answer": "Б" # one of the first 4 cyrillyc letters: "А", "Б", "В" or "Г"
}}


### Question ###
{question}

{option_1}
{option_2}
{option_3}
{option_4}




## Evaluation

In [26]:
def get_response(
    question_dict : Dict, 
    prompt_template: str, 
    system_prompt: str, 
    model="openai/gpt-4o-2024-11-20",
    sampling_params: Dict = {
        "max_tokens": 512,
        "temperature": 0.5,
        "top_p": 0.9
    }
):
    
    options = [
        opt["letter"] + '. ' + opt["option_text"]
        for opt in question_dict["options"]
    ]
    
    prompt = prompt_template.format(
        question=question_dict['question'], 
        option_1=options[0],
        option_2=options[1],
        option_3=options[2], 
        option_4=options[3]
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "{"}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        **sampling_params
    )

    return response

# GPT-4o

In [29]:
results = []
responses = []
for question_dict in tqdm(questions):
    response = get_response(
        question_dict,
        prompt_template,
        system_prompt
    )
    responses.append(response)
    try:
        response_dict = json_repair.loads(response.choices[0].message.content)

        results.append(
            question_dict | {
                "llm_answer" : response_dict["answer"],
                "llm_reasoning" : response_dict["reasoning"]
            }
        )
    except Exception as e:
        results.append({})

  0%|          | 0/44 [00:00<?, ?it/s]

In [42]:
res_df = pd.DataFrame(results).set_index("id")
res_df['correct'] = (res_df.answer == res_df.llm_answer).astype(int)
res_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 1.4 to 11.7
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   question       44 non-null     object
 1   options        44 non-null     object
 2   answer         44 non-null     object
 3   chapter        44 non-null     int64 
 4   llm_answer     44 non-null     object
 5   llm_reasoning  44 non-null     object
 6   correct        44 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 2.8+ KB


In [43]:
res_df.head()

,question,options,answer,chapter,llm_answer,llm_reasoning,correct
id,,,,,,,
1.4,Ликвидность акции характеризует,"[{'letter': 'А', 'option_text': 'Способность и...",А,1,А,"Ликвидность акции определяется тем, насколько ...",1
1.5,Что из перечисленного не является риском по пр...,"[{'letter': 'А', 'option_text': 'Риск изменени...",А,1,А,Риск изменения суверенного рейтинга Российской...,1
1.6,"В фондовый индекс, рассчитываемый биржей, вклю...","[{'letter': 'А', 'option_text': 'Все акции, до...",Б,1,Б,"Фондовый индекс формируется на основе акций, к...",1
1.7,"В случае, если Вы купили иностранную акцию за ...","[{'letter': 'А', 'option_text': '500 рублей.'}...",В,1,В,Для расчета налогооблагаемого дохода в России ...,1
2.4,Вы получили убытки от совершения сделок с акци...,"[{'letter': 'А', 'option_text': 'Нет, не возме...",А,2,А,"Убытки от сделок с акциями не возмещаются, так...",1


In [44]:
res_df.correct.sum(), res_df.correct.mean()

(np.int64(41), np.float64(0.9318181818181818))

In [45]:
res_df.groupby("chapter")['correct'].mean()

chapter
1     1.00
2     0.75
3     1.00
4     0.75
5     1.00
6     1.00
7     1.00
8     1.00
9     1.00
10    0.75
11    1.00
Name: correct, dtype: float64

In [47]:
res_df.to_csv('/Users/alfa/Code/financial_assistant/data/results/gpt4o_zeroshot.csv')

# GPT-4o-mini

In [48]:
results = []
responses = []
for question_dict in tqdm(questions):
    response = get_response(
        question_dict,
        prompt_template,
        system_prompt,
        model="openai/gpt-4o-mini"
    )
    responses.append(response)
    try:
        response_dict = json_repair.loads(response.choices[0].message.content)

        results.append(
            question_dict | {
                "llm_answer" : response_dict["answer"],
                "llm_reasoning" : response_dict["reasoning"]
            }
        )
    except Exception as e:
        results.append({})

  0%|          | 0/44 [00:00<?, ?it/s]

In [56]:
fixed_responses = []
results = []
for q,r in zip(questions, responses):
    r_text =  r.choices[0].message.content
    if '{' not in r_text[:5]:
        r_text = '{' + r_text
    r_json = json_repair.loads(r_text)
    fixed_responses.append(r_json)

    results.append(
        q | {
            "llm_answer" : r_json["answer"],
            "llm_reasoning" : r_json["reasoning"]
        }
    )


In [57]:
res_df = pd.DataFrame(results).set_index("id")
res_df['correct'] = (res_df.answer == res_df.llm_answer).astype(int)
res_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 1.4 to 11.7
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   question       44 non-null     object
 1   options        44 non-null     object
 2   answer         44 non-null     object
 3   chapter        44 non-null     int64 
 4   llm_answer     44 non-null     object
 5   llm_reasoning  44 non-null     object
 6   correct        44 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 2.8+ KB


In [58]:
res_df.head()

,question,options,answer,chapter,llm_answer,llm_reasoning,correct
id,,,,,,,
1.4,Ликвидность акции характеризует,"[{'letter': 'А', 'option_text': 'Способность и...",А,1,А,"Ликвидность акции определяет, насколько легко ...",1
1.5,Что из перечисленного не является риском по пр...,"[{'letter': 'А', 'option_text': 'Риск изменени...",А,1,Г,Все перечисленные риски связаны с приобретение...,0
1.6,"В фондовый индекс, рассчитываемый биржей, вклю...","[{'letter': 'А', 'option_text': 'Все акции, до...",Б,1,Б,"Фондовый индекс формируется на основе акций, к...",1
1.7,"В случае, если Вы купили иностранную акцию за ...","[{'letter': 'А', 'option_text': '500 рублей.'}...",В,1,В,При продаже иностранной акции в рублях необход...,1
2.4,Вы получили убытки от совершения сделок с акци...,"[{'letter': 'А', 'option_text': 'Нет, не возме...",А,2,А,"В общем случае, брокеры не несут ответственнос...",1


In [59]:
res_df.correct.sum(), res_df.correct.mean()

(np.int64(34), np.float64(0.7727272727272727))

In [60]:
res_df.groupby("chapter")['correct'].mean()

chapter
1     0.75
2     0.50
3     1.00
4     0.75
5     1.00
6     0.75
7     0.75
8     1.00
9     0.75
10    0.75
11    0.50
Name: correct, dtype: float64

In [61]:
res_df.to_csv('/Users/alfa/Code/financial_assistant/data/results/gpt4o-mini_zeroshot.csv')